In [53]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
def dtw(x):
    return x%7
def get_month(x):
    if(0<=x<31):
        return 0
    elif(30<x<61):
        return 1
    elif(60<x<92):
        return 2
    if 152<x<184:
        return 3
    elif 183<x<212:
        return 4
    elif  211<x<243:
        return 5
    elif  242<x<273:
        return 6
    elif 272<x<304:
        return 7
    elif 303<x<334:
        return 8
    elif 333<x<365:
        return 9
    elif (364<x<396):
        return 10
    elif (395<x<426):
        return 11
    elif (425<x<457):
        return 12
    elif (91<x<122):
        return 13
    elif 121<x<153:
        return 14
    else:
        return 0
def get_day(x):
    if 152<x<184:
        return x-152
    elif 183<x<212:
        return x-183
    elif  211<x<243:
        return x-211
    elif  242<x<273:
        return x-242
    elif 272<x<304:
        return x-272
    elif 303<x<334:
        return x-303
    elif 333<x<365:
        return x-333
    elif (364<x<396):
        return x-364
    elif(0<=x<31):
        return x+1
    elif (395<x<426):
        return x-395
    elif(30<x<61):
        return x-30
    elif (425<x<457):
        return x-425
    elif(60<x<92):
        return x-60
    elif 121<x<153:
        return x-121
    else:
        return 0
def time(x):
    return x[-8:]

In [3]:
%time transactions = pd.read_csv('data/transactions.csv')

CPU times: user 7.08 s, sys: 508 ms, total: 7.58 s
Wall time: 7.89 s


In [4]:
train_transactions = transactions[transactions.amount < 0].copy()
train_transactions['day'] = train_transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

In [5]:
train_transactions['amount']=-train_transactions.amount

In [ ]:
#del transactions

train_transactions.head()

In [ ]:
train_transactions.shift(1).head()

# VIZUALIZATION

In [ ]:
train_transactions['month'] = train_transactions['day'].apply(get_month)

In [ ]:
plt.scatter(train_transactions[train_transactions['mcc_code']==4814].day,np.log(train_transactions[train_transactions['mcc_code']==4814].amount))

In [ ]:
max(train_transactions['day'])

# TEST

In [6]:
np.arange(1, 31)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [7]:

test_transactions = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + train_transactions.day.max())

In [8]:

test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)


In [9]:
test_transactions.columns = ['mcc_code', 'day']

In [10]:
test_transactions.head()

,mcc_code,day
0,4814,457
1,4814,458
2,4814,459
3,4814,460
4,4814,461


In [11]:
train_grid = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                          index=train_transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

In [12]:
train_grid.head()

,mcc_code,day
0,4814,0
1,4814,1
2,4814,2
3,4814,3
4,4814,4


## 

In [13]:
for tr_table in [train_transactions, test_transactions, train_grid]:
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day']//30#.apply(get_month)# // 30
    tr_table['month_day'] = tr_table['day']%30#.apply(get_day)# % 30

# Statistics

In [14]:
train_transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,day,week_num,week_day,month_num,month_day
0,39026145,0 10:23:26,4814,1030,2245.92,NaN,0,0,0,0,0
2,39026145,1 10:20:56,4829,2330,56147.89,NaN,1,0,1,0,1
3,39026145,1 10:39:54,5499,1010,1392.47,NaN,1,0,1,0,1
4,39026145,2 15:33:42,5499,1010,920.83,NaN,2,0,2,0,2
5,39026145,2 15:53:49,5541,1010,14643.37,NaN,2,0,2,0,2


In [15]:
train_transactions = \
    pd.merge(train_grid,
             train_transactions.groupby(['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code'])[['amount']]\
                 .sum().reset_index(),
             how='left').fillna(0)

In [16]:
#train_transactions['amount'] = -train_transactions.amount
train_transactions.head()

,mcc_code,day,week_num,week_day,month_num,month_day,amount
0,4814,0,0,0,0,0,11098744.26
1,4814,1,0,1,0,1,7881825.53
2,4814,2,0,2,0,2,6777480.45
3,4814,3,0,3,0,3,9277943.73
4,4814,4,0,4,0,4,9999757.21


In [ ]:
train_transactions.head()

In [ ]:
for i in range(14):
    print(np.mean(train_transactions[train_transactions['month']==i+1].amount)/np.mean(train_transactions[train_transactions['month']==i].amount))

In [ ]:
np.mean([1.384754,1.070759,1.537577])

In [ ]:
np.mean([43,37.5,39])

In [17]:
train_transactions.drop(['month'],axis=1,inplace=True)

ValueError: labels ['month'] not contained in axis

In [18]:
data = train_transactions.copy()

In [205]:
np.mean(train_transactions.amount),np.median(train_transactions.amount)

(4125865.9492456163, 50458.085000000006)

In [120]:
#+ назад,-вперед


In [389]:
train_transactions=data.copy()

In [390]:
for day_shift in [-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]:
    
    for month_shift in [1,2]:# train_transactions.month_num.unique()[1:]:#in [1]:#
        
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(train_shift['amount']+500)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

# Validation

In [440]:
features = list(train_transactions.columns)
print(train_transactions.shape)

(84088, 129)


In [441]:
Xval = train_transactions.loc[(train_transactions["month_num"]==15) | ((train_transactions["month_num"]==14)& (train_transactions["month_day"]>6)),features]
print(Xval.shape)
z = Xval.index
Xval.head()

(5520, 129)


,mcc_code,day,week_num,week_day,month_num,month_day,amount,amount_day_-30_1,amount_day_-30_2,amount_day_-29_1,...,amount_day_26_1,amount_day_26_2,amount_day_27_1,amount_day_27_2,amount_day_28_1,amount_day_28_2,amount_day_29_1,amount_day_29_2,amount_day_30_1,amount_day_30_2
294,4814,427,61,0,14,7,13178736.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,4814,428,61,1,14,8,8886759.49,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,4814,429,61,2,14,9,8119948.53,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297,4814,430,61,3,14,10,11046747.74,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298,4814,431,61,4,14,11,11347381.37,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [452]:
Xtrain =  train_transactions.loc[list(set(train_transactions.index)-set(z)),features]
Xtrain = Xtrain[Xtrain['month_num']>6]
print(Xtrain.shape)
Xtrain.head()

(39928, 129)


,mcc_code,day,week_num,week_day,month_num,month_day,amount,amount_day_-30_1,amount_day_-30_2,amount_day_-29_1,...,amount_day_26_1,amount_day_26_2,amount_day_27_1,amount_day_27_2,amount_day_28_1,amount_day_28_2,amount_day_29_1,amount_day_29_2,amount_day_30_1,amount_day_30_2
137,4814,210,30,0,7,0,11807132.98,0,0,16.324982,...,0,0,0,0,0,0,0,0,0,0
138,4814,211,30,1,7,1,10125215.38,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
139,4814,213,30,3,7,3,12183943.03,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
140,4814,214,30,4,7,4,9410543.81,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
141,4814,215,30,5,7,5,8776986.56,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [458]:
Xtrain = pd.get_dummies(Xtrain,columns=['mcc_code'])
Xval = pd.get_dummies(Xval, columns=['mcc_code'])

In [459]:
feat = list(set(Xtrain.columns)-{'amount'})

In [460]:
shift = 500
dtr = xgb.DMatrix(Xtrain[feat], label=np.log(Xtrain['amount'] + shift))
dval = xgb.DMatrix(Xval[feat], label=np.log(Xval['amount'] + shift))
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [464]:
params = {
    'max_depth': 4,
    'eta': 0.05,
    'nthread': 4,
    'subsample': 0.8,
    'eval_metric': ['rmse'],
    'objective': 'reg:linear',
    'colsample_bytree': 0.6,
    'min_child_weight': 0,
    #'scale_pos_weight':(1-train.Label.mean())/train.Label.mean(),
    'seed':7
}

In [470]:
clf.fit(Xtrain[feat], np.log(Xtrain['amount'] + shift))

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=300, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=7, silent=True, subsample=0.8)

In [471]:
pred = clf.predict(Xval[feat])

In [473]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(np.log(Xval['amount'] + shift), pred))

1.5695946841565791

In [466]:
model = xgb.train(params, dtr, num_boost_round=1000, evals=watchlist,evals_result=history, verbose_eval=10)

[0]	train-rmse:10.1008	eval-rmse:10.2415
[10]	train-rmse:6.20794	eval-rmse:6.29242
[20]	train-rmse:3.95031	eval-rmse:3.99684
[30]	train-rmse:2.69781	eval-rmse:2.72401
[40]	train-rmse:2.05944	eval-rmse:2.07579
[50]	train-rmse:1.76629	eval-rmse:1.77982
[60]	train-rmse:1.64308	eval-rmse:1.65889
[70]	train-rmse:1.59203	eval-rmse:1.61245
[80]	train-rmse:1.56884	eval-rmse:1.59431
[90]	train-rmse:1.55676	eval-rmse:1.58625
[100]	train-rmse:1.54912	eval-rmse:1.58239
[110]	train-rmse:1.54338	eval-rmse:1.57981
[120]	train-rmse:1.53847	eval-rmse:1.57784
[130]	train-rmse:1.53435	eval-rmse:1.57712
[140]	train-rmse:1.52987	eval-rmse:1.57486
[150]	train-rmse:1.5261	eval-rmse:1.57228
[160]	train-rmse:1.52233	eval-rmse:1.57245
[170]	train-rmse:1.51848	eval-rmse:1.57222
[180]	train-rmse:1.51484	eval-rmse:1.57178
[190]	train-rmse:1.51161	eval-rmse:1.57103
[200]	train-rmse:1.50823	eval-rmse:1.5703
[210]	train-rmse:1.50479	eval-rmse:1.57054
[220]	train-rmse:1.50188	eval-rmse:1.5708
[230]	train-rmse:1.49873	

KeyboardInterrupt: 

In [427]:
xgb.cv(params, dtr, 1000, nfold=3,
       metrics={'rmse'}, seed = 7,verbose_eval=50,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

[0]	train-rmse:10.1008+0.0113875	test-rmse:10.101+0.0230824
[0]	train-rmse:10.1008+0.0113875	test-rmse:10.101+0.0230824
[1]	train-rmse:9.61242+0.0107226	test-rmse:9.61259+0.0222459
[2]	train-rmse:9.14917+0.0100144	test-rmse:9.14979+0.0219871
[3]	train-rmse:8.70976+0.00940378	test-rmse:8.71064+0.0214687
[4]	train-rmse:8.29298+0.00895601	test-rmse:8.29413+0.0203822
[5]	train-rmse:7.89754+0.00839295	test-rmse:7.89888+0.0196945
[6]	train-rmse:7.52291+0.00776791	test-rmse:7.52439+0.0186747
[7]	train-rmse:7.16728+0.00718229	test-rmse:7.16912+0.0178437
[8]	train-rmse:6.83026+0.00676574	test-rmse:6.83263+0.0168942
[9]	train-rmse:6.51112+0.006373	test-rmse:6.51377+0.0160586
[10]	train-rmse:6.20847+0.00592512	test-rmse:6.21159+0.015198
[11]	train-rmse:5.92181+0.00557228	test-rmse:5.92516+0.0146698
[12]	train-rmse:5.65053+0.0049864	test-rmse:5.65456+0.014093
[13]	train-rmse:5.39391+0.00461742	test-rmse:5.39811+0.0134334
[14]	train-rmse:5.1508+0.00403243	test-rmse:5.15559+0.0127477
[15]	train-rmse

KeyboardInterrupt: 

1602
#1702 no mcc full time
#1687 no 0 minth
#168 >6 month
1.70 >7
1,67>5
1.62 >5 -30
1.623 >5 0
1.64 >0
1.63 nan >6
1/628 nan >5
1621 0 >5 mcc
2 month:
1.623>5
1.619Ю6
0/621 >6 mcc

In [61]:
xgbmodel = xgb.XGBRegressor(max_depth=4, learning_rate=0.025, n_estimators=750, objective='reg:linear', nthread=-1, gamma=0, min_child_weight=1, subsample=0.9, colsample_bytree=0.7, reg_alpha=0, reg_lambda=1, seed=7, missing=-9999)

In [63]:
xgbmodel.fit(Xtrain[feat], Xtrain['amount'] + shift)

KeyboardInterrupt: 

In [48]:
train = pd.get_dummies(train_transactions,columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])

In [49]:
train.head()

,day,week_num,week_day,month_num,month_day,amount,amount_day_0_1,amount_day_1_1,amount_day_2_1,amount_day_3_1,...,mcc_code_8299,mcc_code_8398,mcc_code_8641,mcc_code_8699,mcc_code_8999,mcc_code_9211,mcc_code_9222,mcc_code_9311,mcc_code_9399,mcc_code_9402
0,0,0,0,0,0,11098744.26,-9999,-9999,-9999,-9999,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,7881825.53,-9999,-9999,-9999,-9999,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,2,6777480.45,-9999,-9999,-9999,-9999,...,0,0,0,0,0,0,0,0,0,0
3,3,0,3,0,3,9277943.73,-9999,-9999,-9999,-9999,...,0,0,0,0,0,0,0,0,0,0
4,4,0,4,0,4,9999757.21,-9999,-9999,-9999,-9999,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Xtr = pd.get_dummies(train_transactions[train_transactions['month_num']!=14],columns=['mcc_code'])
Xval = pd.get_dummies(train_transactions[train_transactions['month_num']==14],columns=['mcc_code'])
Xtr.shape,Xval.shape

In [ ]:
Xval = Xval.reset_index(drop=True)

In [ ]:
c = Xtr.columns.difference(['amount'])

In [ ]:
shift = 500
clf = LinearRegression()
clf.fit(Xtr[c], np.log(-Xtr['amount'] + shift))

In [ ]:
p=clf.predict(Xval[c]) - shift

In [ ]:
Xval['amount'][77]

In [ ]:
rmsl500(-Xval['amount'],pval)

In [ ]:
pval = np.e ** clf.predict(Xval[c]) - shift

In [ ]:
(pval)

In [ ]:
abs(-3.1)

In [434]:
train.head()

,day,week_num,week_day,month_num,month_day,amount,amount_day_-30_1,amount_day_-30_2,amount_day_-29_1,amount_day_-29_2,...,mcc_code_8299,mcc_code_8398,mcc_code_8641,mcc_code_8699,mcc_code_8999,mcc_code_9211,mcc_code_9222,mcc_code_9311,mcc_code_9399,mcc_code_9402
137,210,30,0,7,0,11807132.98,0,0,16.324982,16.215874,...,0,0,0,0,0,0,0,0,0,0
138,211,30,1,7,1,10125215.38,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
139,213,30,3,7,3,12183943.03,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
140,214,30,4,7,4,9410543.81,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
141,215,30,5,7,5,8776986.56,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [435]:
shift = 500
train = pd.get_dummies(train_transactions, columns=['mcc_code'])
train = train[train['month_num']>6]
test = pd.get_dummies(test_transactions, columns=['mcc_code'])
c = train.columns.difference(['amount'])

In [ ]:
params = {
    'max_depth': 6,
    'eta': 0.025,
    'nthread': 4,
    'subsample': 0.8,
    'eval_metric': ['rmsl'],
    'colsample_bytree': 0.9,
    'min_child_weight': 0,
    #'scale_pos_weight':(1-train.Label.mean())/train.Label.mean(),
    'seed':1337
}

In [467]:
clf = xgb.XGBRegressor(max_depth=4, learning_rate=0.05, n_estimators=300, objective='reg:linear', nthread=-1, gamma=0, min_child_weight=0, subsample=0.8, colsample_bytree=0.6, reg_alpha=0, reg_lambda=1, seed=7, missing=None)

In [437]:
clf.fit(train[c], np.log(train['amount'] + shift))

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=300, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=7, silent=True, subsample=0.8)

In [ ]:


#clf = LinearRegression()
#clf.fit(train[c], np.log(-train['amount'] + shift))

In [438]:
test_transactions['volume'] = np.e ** clf.predict(test[c]) - shift
test_transactions[['mcc_code', 'day', 'volume']].to_csv('xgbbase-c_nan.csv', index=False)

KeyError: "['month_num_10' 'month_num_11' 'month_num_12' 'month_num_13' 'month_num_14'\n 'month_num_7' 'month_num_8' 'month_num_9' 'week_num_30' 'week_num_31'\n 'week_num_32' 'week_num_33' 'week_num_34' 'week_num_35' 'week_num_36'\n 'week_num_37' 'week_num_38' 'week_num_39' 'week_num_40' 'week_num_41'\n 'week_num_42' 'week_num_43' 'week_num_44' 'week_num_45' 'week_num_46'\n 'week_num_47' 'week_num_48' 'week_num_49' 'week_num_50' 'week_num_51'\n 'week_num_52' 'week_num_53' 'week_num_54' 'week_num_55' 'week_num_56'\n 'week_num_57' 'week_num_58' 'week_num_59' 'week_num_60' 'week_num_61'\n 'week_num_62' 'week_num_63' 'week_num_64'] not in index"

In [ ]:
def rmsl500(y,p):
    z=0
    for i in range(len(y)):
        z = z+(np.log(y[i]+500)-np.log(abs(p[i])+500))**2
    z =z/len(y)
    print(z)
    z = np.sqrt(z)
    return z

In [ ]:
rmsl500